In [ ]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random

from carla import log
from carla import MLModelCatalog
from carla.data.catalog import CsvCatalog, OnlineCatalog, DataCatalog
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods import Dice, Wachter
from copy import deepcopy
from datetime import datetime
from func_timeout import func_timeout, FunctionTimedOut
from ipynb.fs.full.dynamic_recourse import DynamicCsvCatalog, DynamicMLModelCatalog
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = ""

#     torch.manual_seed(0)
#     random.seed(0)
#     np.random.seed(0)

In [ ]:
def k_means(data, min_clusters=1, max_clusters=10):
    """
    Applies the k-means algorithm and automatically estimates the elbow point.
    The algorithm used to calculate the elbow point is described in 10.1109/ICDCSW.2011.20
    
    Args:
        data (pandas.DataFrame): 
            Records along with their labels.
        min_clusters (int): 
            Minimal number of clusters that is expected in the dataset.
        max_clusters (int):
            Maximal number of clusters that is expected in the dataset.
        
    Returns:
        int: Estimated number of clusters that yields the elbow point on an inertia graph.
    """
    clusters = []
    scores = []
    # Fit different potential numbers of clusters
    for k in range(min_clusters, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
        clusters.append(k)
        scores.append(kmeans.inertia_)   
    
    # Automatically find the elbow point, this should change at some point during the application of AR
    # if the counterfactual instances form their own cluster(s), the value returned by this method should change.
    kneedle = KneeLocator(clusters, scores, curve="convex", direction="decreasing")

    return int(kneedle.elbow)

def class_statistics(data, target, aggregate):
    """
    Applies an aggregation function for the two classes described in the dataset.
    
    Args:
        data (pandas.DataFrame):
            Records along with their labels.
        target (str): 
            Name of the target column.
        aggregate (Callable): 
            An aggregation function which can be applied on data.
        
    Returns:
        dict: Values return by the aggregation function applied on the positive and negative class.
    """
    features = data.loc[:, data.columns != target]
    positive_samples = features.loc[data[target] == positive_class]
    negative_samples = features.loc[data[target] == negative_class]
    return {"positive": aggregate(positive_samples).to_dict(), 
            "negative": aggregate(negative_samples).to_dict()}

def measure_distribution(dataset):
    """
    Computes a set of statistical measures for the distribution of a dataset.
    
    Args:
        dataset (DataCatalog):
            Catalog containing a dataframe, set of train and test records, and the target.
        
    Returns:
        dict: Statistics calculated for a specific distribution of data.
    """
    num_clusters = k_means(dataset.df.loc[:, dataset.df.columns != dataset.target].to_numpy())
    means = class_statistics(dataset.df, dataset.target, np.mean)
    stds = class_statistics(dataset.df, dataset.target, np.std)
    
    return {"num_clusters": num_clusters, "means": means, "stds": stds}

In [ ]:
def train_model(dataset, hyper_params, retrain=False):
    """
    Instantiates and trains a black-box model within a CARLA wrapper that will be used to generate explanations.
    
    Args:
        dataset (DataCatalog): 
            Catalog containing a dataframe, set of train and test records, and the target.
        hyper_params (dict): 
            Dictionary storing all custom hyper-parameter values for the model.
        
    Returns:
        MLModelCatalog: 
            Classifier with additional utilities required by CARLA.
    """
    
    model = DynamicMLModelCatalog(dataset, model_type='ann',
                                     load_online=False, backend="pytorch")

    # force_train is enabled to ensure that the model is not reused from the cache
    if not retrain:
        model.train(learning_rate=hyper_params['learning_rate'],
                    epochs=hyper_params['epochs'],
                    batch_size=hyper_params['batch_size'],
                    hidden_size=hyper_params['hidden_size'],
                    force_train=True)
    else:
        model.retrain(learning_rate=hyper_params['learning_rate'],
                      epochs=hyper_params['epochs'],
                      batch_size=hyper_params['batch_size'])
        
    
    return model

In [ ]:
def performance_metrics(dataset, model):
    """
    Computes a set of performance metrics for a classifier.
    
    Args:
        dataset (DataCatalog): 
            Catalog containing a dataframe, set of train and test records, and the target.
        model (MLModelCatalog) 
            Classifier with additional utilities required by CARLA.
    
    Returns:
        tuple of (float, float): Accuracy and F1 score of the model.
    """
    predictions = model.predict_proba(dataset.df_test.loc[:, dataset.df_test.columns != dataset.target])
    predicted_labels = np.argmax(predictions, axis=1) 
    ground_truth = dataset.df_test.loc[:, dataset.df_test.columns == dataset.target].values.astype(int).flatten() 
    
    return accuracy_score(ground_truth, predicted_labels), f1_score(ground_truth, predicted_labels)

In [ ]:
def recourse_worker(generator, factual):
    """
    Apply algorithmic recourse for a (set of) factuals using a chosen generator.
    
    Args:
        generator (RecourseMethod): 
            Generator that finds counterfactual explanations using a black-box model.
        factual (pandas.DataFrame): 
            One or more records from a dataset used to train the black-box model.
        
    Returns:
        pandas.DataFrame: A counterfactual explanation for the provided factual.
    """
    counterfactuals = generator.get_counterfactuals(deepcopy(factual)).dropna()
    if not counterfactuals.empty:
        return counterfactuals.sample().astype(float)
    
def recourse_controller(function, max_wait_time, generator, factual):
    """
    Wrapper function that ensures the application of recourse does not run indefinitely.
    
    Args:
        function (Callable): 
            Function that will have its execution placed under a timeout.
        max_wait_time (int): 
            Number of seconds after which the `function` will time out.
        generator (RecourseMethod): 
            Generator that finds counterfactual explanations using a black-box model.
        factual (pandas.DataFrame): 
            One or more records from a dataset used to train the black-box model.
        
    Returns:
        pandas.DataFrame: A counterfactual explanation for the provided factual if found.
    """
    try:
        return func_timeout(max_wait_time, function, args=[generator, factual]) 
    except FunctionTimedOut:
        log.info("Timeout - No Counterfactual Explanation Found")

    return None

In [ ]:
def plot_distribution(data, output_directory, generator_name, plot_name, plot_id, show_plot=False):
    """
    Generates a plot a two-class dataset and saves it to a file.
    
    Args:
        data (pandas.DataFrame): 
            Records along with their labels.
        output_directory (str): 
            Name of the directory where images are saved.
        generator_name (str): 
            Name of the applied recourse generator.
        plot_name (str): 
            Type of the created plot.
        plot_id (str): 
            ID for the generated plot (e.g. consecutive numbers for different distributions).
        show_plot (Boolean): 
            If True the plot will also be outputted directly to the notebook.
    """
    colors = np.array(['#1f77b4', '#ff7f0e'])
    plt.scatter(data['feature1'], data['feature2'], c=colors[data['target'].astype(int)])
    plt.axis('equal')
    plt.grid(True)
    plt.xlim([-0.2, 1.2])
    plt.ylim([-0.3, 1.3])
    plt.suptitle(f'Recourse generated by {generator_name.upper()} at t = {plot_id}')
    plt.savefig(f'{output_directory}/{plot_name}_{plot_id}.png', bbox_inches='tight')
    if show_plot:
        plt.show()
    plt.close()

In [ ]:
# TODO: Generalize code and names
# TODO: Split this into more code blocks for readability (what would be a good way?)
# TODO: What to do if a generator times out? do we accept different numbers of samples?
def run_experiment(dataset, hyper_params, epochs=0.5, recourse_per_epoch=0.01,
                   generator_timeout=120, generator_params={}, benchmark_params={}):
    """
    Driver code to execute an experiment that allows to compare the dynamics of recourse 
    applied by some generator to a benchmark described by Wachter et al. (2017)
    
    Args:
        dataset (DataCatalog): 
            Catalog containing a dataframe, set of train and test records, and the target.
        hyper_params (dict): 
            Dictionary storing all custom hyper-parameter values for the model.
        epochs (float): 
            Value between 0 and 1 representing the proportion of samples from the training set
            which should have recourse applied to them throughout the experiment.
        recourse_per_epoch (float): 
            Value between 0 and 1 representing the proportion of samples from the training set
            which should have recourse applied to them in a single iteration.
        generator_timeout (int): 
            Number of seconds after which the search for counterfactuals will time out.
        generator_params (dict): 
            Dictionary representing the hyper-parameters for the generator under test (DICE).
        benchmark_params (dict): 
            Dictionary representing the hyper-parameters for the benchmark generator (Wachter).
    """
     
    # Experiment data is saved into a new directory
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    os.makedirs(f'../experiment_data/{timestamp}')
    
    # 1. Measure the initial distribution of data
    experiment_data = {'dice': {0: {}}, 'wachter': {0: {}}}
    experiment_data['dice'][0]['distribution'] = measure_distribution(dataset)
    experiment_data['wachter'][0]['distribution'] = measure_distribution(dataset)
    
    # 2. Train classifiers C1, C2 on the dataset (one for DICE, one for benchmarking with Wachter)
    log.info("Training the initial model")
    model = train_model(dataset, hyper_params)
    
    # Recourse generated by DICE is compared with the Wachter generator, as they may modify data differently
    # we need to keep track of two models and two datasets and update them independently
    benchmark_model = deepcopy(model)
    benchmark_dataset = deepcopy(dataset)
    
    # 3. Measure the performance of the initial models
    acc_dice, f1_dice = performance_metrics(dataset, model)
    experiment_data['dice'][0]["performance"] = {"acc": acc_dice, "f1": f1_dice}
    
    acc_wachter, f1_wachter = performance_metrics(benchmark_dataset, benchmark_model)
    experiment_data['wachter'][0]["performance"] = {"acc": acc_dice, "f1": f1_dice}
     
    # factuals are a list of instances that the model expects to belong to the negative class;
    # in order to acurately measure the performance of the dataset we never change the test set
    factuals = predict_negative_instances(model, dataset.df_train)
    factuals_index = factuals.index.tolist()
    
    # TODO: make this slightly more robust
    recourse_per_epoch = max(int(recourse_per_epoch * len(factuals)), 1)
    epochs = max(int(min(epochs, 1) * len(factuals) / recourse_per_epoch), 1)

    # Instantiate recourse generators
    dice = Dice(model, generator_params)
    wachter = Wachter(benchmark_model, benchmark_params)
    
    plot_distribution(dataset._df, f'../experiment_data/{timestamp}', "dice", "distribution", 0)
    plot_distribution(benchmark_dataset._df, f'../experiment_data/{timestamp}', "wachter", "distribution", 0)

    for epoch in range(epochs - 1):
        log.info(f"Starting epoch {epoch + 1}")
        experiment_data['dice'][epoch + 1] = {}
        experiment_data['wachter'][epoch + 1] = {}
        # 4. Generate a subset S of factuals that have never been encountered by the generators
        current_factuals_index = np.random.choice(factuals_index, replace=False, size=recourse_per_epoch)
        current_factuals = dataset._df.iloc[current_factuals_index]
        # We do not want to accidentally generate a counterfactual from a counterfactual
        factuals_index = [f for f in factuals_index if f not in current_factuals_index]  
        
        # 5. Apply recourse for S with DICE
        log.info(f"Applying DICE")
        dice_counterfactuals_found = 0
        # We have to modify factuals one by one because CARLA outputs diverse counterfactuals 
        # generated for different factuals in a single dataframe
        for i in range(len(current_factuals)):
            f = current_factuals.iloc[[i]]
            
            log.info(f"Generating counterfactual {i + 1} with DICE")
            # CARLA does not implement a timeout for the generators by default
            # but we want to prevent the code from running indefinitely
            dice_counterfactuals = recourse_controller(recourse_worker, generator_timeout, dice, f)
            # We only want to overwrite the existing data if counterfactual generation was successful
            if dice_counterfactuals is not None and not dice_counterfactuals.empty:
                dataset._df.iloc[f.index[0]] = dice_counterfactuals.iloc[0]
                dice_counterfactuals_found += 1
        experiment_data['dice'][epoch + 1]['counterfactuals'] = dice_counterfactuals_found
        
        model.data._df.update(dataset._df)
        model.data._df_train.update(dataset._df)
        
        # 6. Apply recourse for S with Wachter
        log.info(f"Applying the Wachter generator")
        wachter_counterfactuals = wachter.get_counterfactuals(current_factuals).dropna()
        benchmark_dataset._df.update(wachter_counterfactuals)
        experiment_data['wachter'][epoch + 1]['counterfactuals'] = len(wachter_counterfactuals.index)
        
        benchmark_model.data._df.update(benchmark_dataset._df)
        benchmark_model.data._df_train.update(benchmark_dataset._df)

        # 7. Measure the quality of recourse
        
        # 9. Retrain C1, C2 on the updated datasets
        log.info("Updating the DICE model")
        model = train_model(dataset, hyper_params, retrain=True)
        
        log.info("Updating the benchmark model")
        benchmark_model = train_model(benchmark_dataset, hyper_params, retrain=True)
        
        # 8. Measure the resulting distributions of data
        experiment_data['dice'][epoch + 1]['distribution'] = measure_distribution(dataset)
        experiment_data['wachter'][epoch + 1]['distribution'] = measure_distribution(benchmark_dataset)
        
        # 10. Measure the performance of updated models
        acc_dice, f1_dice = performance_metrics(dataset, model)
        experiment_data['dice'][epoch + 1]["performance"] = {"acc": acc_dice, "f1": f1_dice}

        acc_wachter, f1_wachter = performance_metrics(dataset, model)
        experiment_data['wachter'][epoch + 1]["performance"] = {"acc": acc_dice, "f1": f1_dice}
        
        plot_distribution(dataset._df, f'../experiment_data/{timestamp}', "dice", "distribution", epoch + 1)
        plot_distribution(benchmark_dataset._df, f'../experiment_data/{timestamp}', "wachter", "distribution", epoch + 1)
    
    # 11. Output results
    with open(f"../experiment_data/{timestamp}/measurements.json", 'w') as outfile:
        json.dump(experiment_data, outfile, sort_keys=True, indent=4)

In [ ]:
# Load the dataset from a csv file
dataset = DynamicCsvCatalog(file_path="../datasets/unimodal_dataset_1.csv", 
                            continuous=['feature1', 'feature2'], categorical=[],
                            immutables=[], target='target', test_size=0.2)

# Encoding of the classes applied in the dataset
positive_class = 1
negative_class = 0

In [ ]:
hyper_params = {'learning_rate': 0.04, 'epochs': 3, 'batch_size': 1, 'hidden_size': [4]}
dice_params = {"num": 3}
wachter_params = {"loss_type": "BCE", "t_max_min": 0.1}
run_experiment(dataset, hyper_params, epochs=0.2, recourse_per_epoch=0.01,
               generator_timeout=20, generator_params=dice_params, benchmark_params=wachter_params)